In [1]:
import sys

In [4]:
sys.path.append("../../")

In [5]:
from CCG import utils
from CCG import constant
from nltk.ccg import chart, lexicon
import re

In [6]:
mini_sent = """
The phrase \"'s grandmother\" occurs between SUBJ and OBJ
"""

mini_sent_2 = """
SUBJ is left of OBJ
"""

longer_sent = """
The phrase \"'s grandmother\" occurs between SUBJ and OBJ and there are no more than four words between SUBJ and OBJ .
"""

In [7]:
sent = longer_sent

In [8]:
pre_process = utils.pre_process_sent(sent)

In [9]:
sent_tokenized = utils.print_tokenized(pre_process)

In [10]:
sent_tokenized

[['$The',
  '$Word',
  '"\'s grandmother"',
  '$Is',
  '$Between',
  '$ArgX',
  '$And',
  '$ArgY',
  '$And',
  '$There',
  '$Is',
  '$AtMost',
  "'4'",
  '$Word',
  '$Between',
  '$ArgX',
  '$And',
  '$ArgY']]

In [11]:
raw_lexicon = constant.raw_lexicon
predicates = utils.new_predicate(sent_tokenized)
raw_lexicon = utils.add_new_predicate(predicates, raw_lexicon)

['$The', '$Word', '"\'s grandmother"', '$Is', '$Between', '$ArgX', '$And', '$ArgY', '$And', '$There', '$Is', '$AtMost', "'4'", '$Word', '$Between', '$ArgX', '$And', '$ArgY']


In [12]:
lex = lexicon.fromstring(raw_lexicon, True)
parser = chart.CCGChartParser(lex, chart.DefaultRuleSet)

In [13]:
for index_1 in range(len(sent_tokenized)):
    while [] in sent_tokenized[index_1]:
        sent_tokenized[index_1].remove([])
    for index_2 in range(len(sent_tokenized[index_1])):
        if sent_tokenized[index_1][index_2].startswith("'") or sent_tokenized[index_1][index_2].startswith("\""):
            sent_tokenized[index_1][index_2] = sent_tokenized[index_1][index_2].replace(' ','SPACE')
            sent_tokenized[index_1][index_2] = sent_tokenized[index_1][index_2].replace("\"","'")

In [14]:
sent_tokenized

[['$The',
  '$Word',
  "''sSPACEgrandmother'",
  '$Is',
  '$Between',
  '$ArgX',
  '$And',
  '$ArgY',
  '$And',
  '$There',
  '$Is',
  '$AtMost',
  "'4'",
  '$Word',
  '$Between',
  '$ArgX',
  '$And',
  '$ArgY']]

In [15]:
logic_forms = []

In [16]:
for i, e in enumerate(sent_tokenized):
    print(i)
    try:
        parses = list(parser.parse(e))
        logic_forms += parses
    except:
        continue

0


In [17]:
len(logic_forms)

3888

In [42]:
raw_sems = len(set([str(tree.label()[0].semantics()) for tree in logic_forms]))

In [43]:
raw_sems

2

In [19]:
raw_sems[0]

"'@And'('@Is'('@And'('There','ArgY'),'@AtMost'('@between'('@And'('ArgY','ArgX')),'@Num'('4','tokens'))),'@Is'('@Word'(''sSPACEgrandmother'),'@between'('ArgX')))"

In [20]:
sem_split = re.split(',|(\()', raw_sems[0])

In [21]:
sem_split = [token for token in sem_split if token]

In [22]:
new_sem = []
for i,w in enumerate(sem_split):
    if w=='(':
        new_sem.insert(-1,'(')
    else:
        new_sem.append(sem_split[i])

In [23]:
for duo in zip(sem_split, new_sem):
    print(duo)

("'@And'", '(')
('(', "'@And'")
("'@Is'", '(')
('(', "'@Is'")
("'@And'", '(')
('(', "'@And'")
("'There'", "'There'")
("'ArgY')", "'ArgY')")
("'@AtMost'", '(')
('(', "'@AtMost'")
("'@between'", '(')
('(', "'@between'")
("'@And'", '(')
('(', "'@And'")
("'ArgY'", "'ArgY'")
("'ArgX'))", "'ArgX'))")
("'@Num'", '(')
('(', "'@Num'")
("'4'", "'4'")
("'tokens')))", "'tokens')))")
("'@Is'", '(')
('(', "'@Is'")
("'@Word'", '(')
('(', "'@Word'")
("''sSPACEgrandmother')", "''sSPACEgrandmother')")
("'@between'", '(')
('(', "'@between'")
("'ArgX')))", "'ArgX')))")


In [24]:
print(new_sem)

['(', "'@And'", '(', "'@Is'", '(', "'@And'", "'There'", "'ArgY')", '(', "'@AtMost'", '(', "'@between'", '(', "'@And'", "'ArgY'", "'ArgX'))", '(', "'@Num'", "'4'", "'tokens')))", '(', "'@Is'", '(', "'@Word'", "''sSPACEgrandmother')", '(', "'@between'", "'ArgX')))"]


In [25]:
s = new_sem[6]
print(s)
sem_replace = s.replace('SPACE', ' ')
for com in constant.comma_index:
    sem_replace = sem_replace.replace('COMMA' + str(constant.comma_index[com]), com)
if sem_replace.startswith("'"):
    if sem_replace.endswith(")"):
        posi = len(sem_replace)-1
        while sem_replace[posi]==")":
            posi-=1
        assert sem_replace[posi]=="\'"
    else:
        posi = len(sem_replace)-1
        assert sem_replace[posi] == "\'"
    print(sem_replace)
    sem_replace = sem_replace[0]+sem_replace[1:posi].replace('\'','\\\'')+sem_replace[posi:]

'There'
'There'


In [26]:
sem_replace

"'There'"

In [27]:
sem_ = ''
for i,s in enumerate(new_sem):
    sem_replace = s.replace('SPACE', ' ')
    for com in constant.comma_index:
        sem_replace = sem_replace.replace('COMMA' + str(constant.comma_index[com]), com)
    if sem_replace.startswith("'"):
        if sem_replace.endswith(")"):
            posi = len(sem_replace)-1
            while sem_replace[posi]==")":
                posi-=1
            assert sem_replace[posi]=="\'"
        else:
            posi = len(sem_replace)-1
            assert sem_replace[posi] == "\'"

        sem_replace = sem_replace[0]+sem_replace[1:posi].replace('\'','\\\'')+sem_replace[posi:]
    if new_sem[i-1]=='(':
        sem_=sem_+sem_replace
    else:
        sem_ = sem_+','+sem_replace

In [29]:
sem_

",('@And',('@Is',('@And','There','ArgY'),('@AtMost',('@between',('@And','ArgY','ArgX')),('@Num','4','tokens'))),('@Is',('@Word','\\'s grandmother'),('@between','ArgX')))"

In [30]:
temp = (".root", eval(sem_[1:]))

In [31]:
hehe = { temp : 1}

In [32]:
temp in hehe

True

In [33]:
parsed_sems = [utils.parse_sem(str(tree.label()[0].semantics())) for tree in logic_forms]

In [34]:
parsed_sems[0]

('.root',
 ('@And',
  ('@Is',
   ('@And', 'There', 'ArgY'),
   ('@AtMost',
    ('@between', ('@And', 'ArgY', 'ArgX')),
    ('@Num', '4', 'tokens'))),
  ('@Is', ('@Word', "'s grandmother"), ('@between', 'ArgX'))))

In [35]:
parsed_sems[1]

('.root',
 ('@And',
  ('@Is',
   ('@And', 'There', 'ArgY'),
   ('@AtMost',
    ('@between', ('@And', 'ArgY', 'ArgX')),
    ('@Num', '4', 'tokens'))),
  ('@Is', ('@Word', "'s grandmother"), ('@between', 'ArgX'))))

In [36]:
type(parsed_sems[1]), len(parsed_sems[1]), len(parsed_sems[1][0]), len(parsed_sems[1][1])

(tuple, 2, 5, 3)

In [37]:
parsed_sems[1][0], type(parsed_sems[1][1])

('.root', tuple)

In [38]:
parsed_sems[1]

('.root',
 ('@And',
  ('@Is',
   ('@And', 'There', 'ArgY'),
   ('@AtMost',
    ('@between', ('@And', 'ArgY', 'ArgX')),
    ('@Num', '4', 'tokens'))),
  ('@Is', ('@Word', "'s grandmother"), ('@between', 'ArgX'))))

In [39]:
parsed_sems[1][1]

('@And',
 ('@Is',
  ('@And', 'There', 'ArgY'),
  ('@AtMost',
   ('@between', ('@And', 'ArgY', 'ArgX')),
   ('@Num', '4', 'tokens'))),
 ('@Is', ('@Word', "'s grandmother"), ('@between', 'ArgX')))

In [40]:
parsed_sems[1][1][1]

('@Is',
 ('@And', 'There', 'ArgY'),
 ('@AtMost', ('@between', ('@And', 'ArgY', 'ArgX')), ('@Num', '4', 'tokens')))

In [41]:
parsed_sems[1][1][2]

('@Is', ('@Word', "'s grandmother"), ('@between', 'ArgX'))

In [39]:
labeling_functions = [utils.recurse(parsed_sem) for parsed_sem in parsed_sems]

In [40]:
labeling_functions[0]()

TypeError: <lambda>() missing 1 required positional argument: 'c'

In [41]:
parsed_sems[0]

('.root',
 ('@And',
  ('@Is',
   ('@And', 'There', 'ArgY'),
   ('@AtMost',
    ('@between', ('@And', 'ArgY', 'ArgX')),
    ('@Num', '4', 'tokens'))),
  ('@Is', ('@Word', "'s grandmother"), ('@between', 'ArgX'))))

In [42]:
def collect_features(semantics, features):
    if isinstance(semantics, tuple):

        for child in semantics[1:]:
            if isinstance(child, tuple) and child[0] != semantics[0]:
                features.append((semantics[0],child[0]))
            collect_features(child, features)

In [43]:
opsfeature = {'@And@Is': 0, '@Is@Word': 1, '@Is@Direct': 2, '@Direct@Left0': 3, '.root@And': 4, '@Is@between': 5, '@Direct@Right0': 6, '@Is@And': 7, '@And@Word': 8, '@between@And': 9, '@Word@And': 10, '@Left0@Word': 11, '@Is@Left0': 12, '@Right0@Word': 13, '@between@Word': 14, '.root@Is': 15, '@Is@Right0': 16, '@Right0@And': 17, '@Left0@And': 18, '@Is@LessThan': 19, '@LessThan@between': 20, '@LessThan@Num': 21, '@And@Right0': 22, '@And@between': 23, '.root@between': 24, '@between@Is': 25, '.root@Left': 26, '@Left@Word': 27, '@And@Left0': 28}


In [44]:
import numpy as np

In [45]:
feature_vector = np.zeros([len(opsfeature),1],np.float32)

In [46]:
op_feature = []

In [47]:
collect_features(parsed_sems[0], op_feature)

In [48]:
op_feature

[('.root', '@And'),
 ('@And', '@Is'),
 ('@Is', '@And'),
 ('@Is', '@AtMost'),
 ('@AtMost', '@between'),
 ('@between', '@And'),
 ('@AtMost', '@Num'),
 ('@And', '@Is'),
 ('@Is', '@Word'),
 ('@Is', '@between')]